In [2]:

%load_ext autoreload
%autoreload 2
from pathlib import Path
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import xarray as xr
import seaborn as sns
import h5py as hf
from tqdm import tqdm
import itertools 
from scipy.interpolate import interp1d
from scipy import signal
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors
import os, fnmatch
from scipy.spatial.distance import cdist
import matplotlib.gridspec as gridspec
from random import sample
from scipy.ndimage import gaussian_filter1d
from matplotlib.collections import LineCollection





import warnings
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
"""load data  """

df = pd.read_hdf(r"D:\obstacle_avoidance\metadata\compiled_data\h5\conditions0.h5")  

In [5]:
df

,level_0,index,first_poke,second_poke,trial_timestamps,trial_vidframes,nose_x,nose_y,leftear_x,leftear_y,...,std_interp_ts_nose_y_cm,mad_interp_ts_nose_y_cm,animal_mean_interp_ts_nose_y_cm,animal_median_interp_ts_nose_y_cm,animal_std_interp_ts_nose_y_cm,animal_mad_interp_ts_nose_y_cm,date_mean_interp_ts_nose_y_cm,date_median_interp_ts_nose_y_cm,date_std_interp_ts_nose_y_cm,date_mad_interp_ts_nose_y_cm
0,0,0,60956.057548,60958.986316,"[60956.064755, 60956.081574, 60956.098918, 609...","[74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 8...","[98.219970703125, 98.28543853759766, 97.297500...","[258.87322998046875, 258.2541198730469, 257.01...","[107.9310073852539, 107.84115600585938, 107.66...","[266.6334533691406, 266.1789245605469, 265.581...",...,"[1.704215957581921, 1.7213475879672797, 1.7144...","[0.8372520181144996, 0.8399219579395094, 0.846...","[14.798852080643828, 14.791549217630285, 14.78...","[15.018471608027324, 15.018471608027324, 15.01...","[1.6374928205447663, 1.6281039149045344, 1.623...","[0.7211336716399082, 0.7211336716399082, 0.718...","[12.895512146474108, 12.895512146474108, 12.89...","[12.743317597940582, 12.743317597940582, 12.74...","[1.5638462442158316, 1.5638462442158316, 1.563...","[1.6199748059719585, 1.6199748059719585, 1.619..."
1,1,2,60965.491417,60967.347059,"[60965.501145, 60965.518067, 60965.535436, 609...","[640, 641, 642, 643, 644, 645, 646, 647, 648, ...","[96.29674530029297, 95.56669616699219, 95.6471...","[254.7836456298828, 254.9083709716797, 255.394...","[101.49925231933594, 100.67830657958984, 100.9...","[262.8636169433594, 263.4535217285156, 263.921...",...,"[1.704215957581921, 1.7213475879672797, 1.7144...","[0.8372520181144996, 0.8399219579395094, 0.846...","[14.798852080643828, 14.791549217630285, 14.78...","[15.018471608027324, 15.018471608027324, 15.01...","[1.6374928205447663, 1.6281039149045344, 1.623...","[0.7211336716399082, 0.7211336716399082, 0.718...","[12.895512146474108, 12.895512146474108, 12.89...","[12.743317597940582, 12.743317597940582, 12.74...","[1.5638462442158316, 1.5638462442158316, 1.563...","[1.6199748059719585, 1.6199748059719585, 1.619..."
2,2,3,60967.347059,60970.579136,"[60967.352576, 60967.368652, 60967.385523, 609...","[[751, 752, 753, 754, 755, 756, 757, 758, 759,...","[675.3671875, 676.18701171875, 676.38055419921...","[267.5236511230469, 267.6513671875, 268.050109...","[667.2891235351562, 669.3956298828125, 669.599...","[254.7681121826172, 253.37440490722656, 253.36...",...,"[2.059087435362894, 2.3323731770032605, 2.6018...","[0.7555298274304194, 0.7493103776097829, 0.750...","[15.361778135635143, 14.884789345270752, 14.44...","[15.544176064562524, 14.97069462834811, 14.410...","[1.2502013049547718, 1.2431528788814532, 1.230...","[0.6910708057772261, 0.6924258996811314, 0.701...","[15.838320421964534, 15.349389731515467, 14.90...","[16.2656834341855, 15.793305962965428, 15.3615...","[1.299432525562957, 1.3478738351732067, 1.3825...","[0.2933357725792902, 0.41185092056603345, 0.51..."
3,3,4,60970.579136,60972.869043,"[60970.586316, 60970.603212, 60970.621478, 609...","[945, 946, 947, 948, 949, 950, 951, 952, 953, ...","[94.26691436767578, 92.74860382080078, 91.8231...","[253.4375762939453, 254.07521057128906, 254.41...","[110.97284698486328, 109.15727996826172, 107.8...","[249.62869262695312, 249.5487060546875, 249.75...",...,"[1.542163282052802, 1.5262591466937574, 1.5398...","[1.0412175161628703, 1.0421551748675064, 1.054...","[25.746848650521077, 25.95367279628537, 25.925...","[25.893985995615203, 26.14472184000985, 26.144...","[1.336183967667448, 1.305419361341502, 1.27439...","[0.8872928491756529, 0.7315714812636696, 0.731...","[24.53565046668779, 24.53565046668779, 24.5356...","[24.53565046668779, 24.53565046668779, 24.5356...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,5,60972.869043,60977.817254,"[60972.871219, 60972.887488, 60972.904, 60972....","[[1082, 1083, 1084, 1085, 1086, 1087, 

In [12]:
df.condition.unique()

array(['whitenoise', 'blackwhite', 'noisewhite', 'whitewhite',
       'whiteblack'], dtype=object)

In [13]:
points = [i for i in df.columns if '_cm' in i]

In [14]:
points

['nose_x_cm',
 'nose_y_cm',
 'leftear_x_cm',
 'leftear_y_cm',
 'rightear_x_cm',
 'rightear_y_cm',
 'spine_x_cm',
 'spine_y_cm',
 'midspine_x_cm',
 'midspine_y_cm',
 'tailbase_x_cm',
 'tailbase_y_cm',
 'midtail_x_cm',
 'midtail_y_cm',
 'tailend_x_cm',
 'tailend_y_cm',
 'arenaTL_x_cm',
 'arenaTL_y_cm',
 'arenaTR_x_cm',
 'arenaTR_y_cm',
 'arenaBL_x_cm',
 'arenaBL_y_cm',
 'arenaBR_x_cm',
 'arenaBR_y_cm',
 'obstacleTL_x_cm',
 'obstacleTL_y_cm',
 'obstacleTR_x_cm',
 'obstacleTR_y_cm',
 'obstacleBR_x_cm',
 'obstacleBR_y_cm',
 'obstacleBL_x_cm',
 'obstacleBL_y_cm',
 'leftportT_x_cm',
 'leftportT_y_cm',
 'leftportB_x_cm',
 'leftportB_y_cm',
 'rightportT_x_cm',
 'rightportT_y_cm',
 'rightportB_x_cm',
 'rightportB_y_cm',
 'ts_nose_x_cm',
 'ts_nose_y_cm',
 'ts_leftear_x_cm',
 'ts_leftear_y_cm',
 'ts_rightear_x_cm',
 'ts_rightear_y_cm',
 'ts_spine_x_cm',
 'ts_spine_y_cm',
 'ts_midspine_x_cm',
 'ts_midspine_y_cm',
 'ts_tailbase_x_cm',
 'ts_tailbase_y_cm',
 'gt_obstacleTL_x_cm',
 'gt_obstacleTL_y_cm'